In [1]:
#notebook based on zflemings:https://nbviewer.jupyter.org/github/zflamig/dask-era5/blob/main/notebook/era5_fargate_dask.ipynb
import xarray as xr
import fsspec
import dask
import s3fs
import numpy as np
xr.set_options(display_style="html")  #display dataset nicely 

In [2]:
import gcsfs
import fsspec
url = 's3://era5-pds/1980/01/data/sea_surface_temperature.nc'
ncfile = fsspec.open(url)
ds = xr.open_dataset(ncfile.open())
ds = ds.mean({'time0'})

PermissionError: Forbidden

In [ ]:
ds

In [ ]:
ds

In [ ]:
years,months=[],[]
[years.append(str(y)) for y in np.arange(1979,2020)]
[months.append(str(m).zfill(2)) for m in np.arange(1,13)]

file_pattern = 's3://era5-pds/{year}/{month}/data/sea_surface_temperature.nc' #air_temperature_at_2_metres.nc'
fnames = [file_pattern.format(year=year,month=month) for year in years for month in months]


In [3]:
years,months=[],[]
[years.append(str(y)) for y in np.arange(1979,2020)]
[months.append(str(m).zfill(2)) for m in np.arange(1,13)]

file_pattern = 's3://era5-pds/{year}/{month}/data/sea_surface_temperature.nc' #air_temperature_at_2_metres.nc'

@dask.delayed
def s3open(path):
    fs = s3fs.S3FileSystem(anon=True, default_fill_cache=False)
    return fs.open(path)

files_mapper = [s3open(file_pattern.format(year=year,month=month)) for year in years for month in months]

In [4]:
fname = files_mapper[0]
ds = xr.open_dataset(fname,engine='h5netcdf')
#ds = ds.mean('time0')
ds

TypeError: Truth of Delayed objects is not supported

In [5]:
ds_av=[]
for fname in files_mapper:
    ds = xr.open_dataset(fname,engine='h5netcdf')
    ds = ds.mean('time0')
    ds_av.append(ds)

TypeError: Truth of Delayed objects is not supported

In [ ]:
ds = xr.open_mfdataset(files_mapper, engine='h5netcdf', chunks={'lon':200,'lat':200,'time0':720}, concat_dim='time1', combine='nested', coords='minimal', compat='override', parallel=True)

ds

In [ ]:
ds